In [1]:
import sys

sys.path.append('/root/lib/')

import ROOT as r
import os
import json
import pandas as pd
import uproot 
import awkward as ak
import array as arr
import numpy as np
import sys

sys.path.append(os.getcwd() + '/../utilities/')
from milliqanProcessor import *
from milliqanScheduler import *
from milliqanCuts import *
from milliqanPlotter import *

%jsroot on
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.30/02


In [2]:
#get list of files to look at
files = []

dataDir = '/store/user/milliqan/trees/v34/1400/'
for filename in os.listdir(dataDir):
    #if len(files) > 10: break
    if not filename.endswith('root'): continue
    if not int(filename.split('Run')[1].split('.')[0]) >= 1400: continue
    files.append(dataDir+filename)

files

['/store/user/milliqan/trees/v34/1400/MilliQan_Run1400.1_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1400.2_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1400.3_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1400.4_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1400.5_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1400.6_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1400.7_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.100_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.101_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.102_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.103_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.104_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.105_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.106_v34.root',
 '/store/user/milliqan/trees/v34/1

In [3]:
@mqCut
def countTriggers(self, cutName=None, trigNum=2):
    triggers = ak.firsts(self.events['tTrigger'])
    binary_trig = 1 << (trigNum-1)
    #print("Binary", trigNum, binary_trig)
    thisTrig = ak.count(triggers[triggers == binary_trig], axis=None)
    #print("instances of trigger {}, {}".format(trigNum, thisTrig))
    self.events[cutName] = thisTrig

print(countTriggers.__name__)

countTriggers


In [4]:
@mqCut
def firstEvent(self):
    events = ak.firsts(self.events['event'])
    mask = np.zeros(len(events), dtype=bool)
    mask[0] = True
    mask = ak.Array(mask)
    self.events['firsts'] = mask

In [8]:
#define a file list to run over
filelist = files[100:110]

print(filelist)

#define the necessary branches to run over
branches = ['event', 'tTrigger', 'boardsMatched', 'pickupFlag', 'fileNumber', 'runNumber']

#define the milliqan cuts object
mycuts = milliqanCuts()

#require pulses are not pickup
pickupCut = mycuts.getCut(mycuts.pickupCut, 'pickupCut', cut=True, branches=branches)

#require that all digitizer boards are matched
boardMatchCut = mycuts.getCut(mycuts.boardsMatched, 'boardMatchCut', cut=True, branches=branches)

setattr(milliqanCuts, 'countTriggers', countTriggers)
setattr(milliqanCuts, 'firstEvent', firstEvent)

@mqCuts("trig1")
trig1 = mycuts.getCut(mycuts.countTriggers, "trig1", trigNum=1)

trig2 = mycuts.getCut(mycuts.countTriggers, "trig2", trigNum=2)
trig3 = mycuts.getCut(mycuts.countTriggers, "trig3", trigNum=3)
trig4 = mycuts.getCut(mycuts.countTriggers, "trig4", trigNum=4)
trig5 = mycuts.getCut(mycuts.countTriggers, "trig5", trigNum=5)
trig7 = mycuts.getCut(mycuts.countTriggers, "trig7", trigNum=7)
trig9 = mycuts.getCut(mycuts.countTriggers, "trig9", trigNum=9)
trig10 = mycuts.getCut(mycuts.countTriggers, "trig10", trigNum=10)
trig11 = mycuts.getCut(mycuts.countTriggers, "trig11", trigNum=11)
trig13 = mycuts.getCut(mycuts.countTriggers, "trig13", trigNum=13)

setattr(milliqanCuts, 'trig1', trig1)

#define milliqan plotter
myplotter = milliqanPlotter()

#create root histogram 
bins = 200
xmin = 0
xmax = 400

h_triggers = r.TH1F("h_triggers", "Triggers", bins, xmin, xmax)
h_trig1 = r.TH1F("h_trig1", "Trigger 1", bins, xmin, xmax)
h_trig2 = r.TH1F("h_trig2", "Trigger 2", bins, xmin, xmax)
h_trig3 = r.TH1F("h_trig3", "Trigger 3", bins, xmin, xmax)
h_trig4 = r.TH1F("h_trig4", "Trigger 4", bins, xmin, xmax)
h_trig5 = r.TH1F("h_trig5", "Trigger 5", bins, xmin, xmax)
h_trig7 = r.TH1F("h_trig7", "Trigger 7", bins, xmin, xmax)
h_trig9 = r.TH1F("h_trig9", "Trigger 9", bins, xmin, xmax)
h_trig10 = r.TH1F("h_trig10", "Trigger 10", bins, xmin, xmax)
h_trig11 = r.TH1F("h_trig11", "Trigger 11", bins, xmin, xmax)
h_trig13 = r.TH1F("h_trig13", "Trigger 13", bins, xmin, xmax)

binsx = 100
xmax_2d = 1500
xmin_2d = 1400
binsy = 200
ymin = 0
ymax = 400

h_trig1_file = r.TH2F("h_trig1_file", "Trigger 1 vs File", binsx, xmin_2d, xmax_2d, binsy, ymin, ymax)
h_trig2_file = r.TH2F("h_trig2_file", "Trigger 2 vs File", binsx, xmin_2d, xmax_2d, binsy, ymin, ymax)
h_trig3_file = r.TH2F("h_trig3_file", "Trigger 3 vs File", binsx, xmin_2d, xmax_2d, binsy, ymin, ymax)
h_trig4_file = r.TH2F("h_trig4_file", "Trigger 4 vs File", binsx, xmin_2d, xmax_2d, binsy, ymin, ymax)
h_trig5_file = r.TH2F("h_trig5_file", "Trigger 5 vs File", binsx, xmin_2d, xmax_2d, binsy, ymin, ymax)
h_trig7_file = r.TH2F("h_trig7_file", "Trigger 7 vs File", binsx, xmin_2d, xmax_2d, binsy, ymin, ymax)
h_trig9_file = r.TH2F("h_trig9_file", "Trigger 9 vs File", binsx, xmin_2d, xmax_2d, binsy, ymin, ymax)
h_trig10_file = r.TH2F("h_trig10_file", "Trigger 10 vs File", binsx, xmin_2d, xmax_2d, binsy, ymin, ymax)
h_trig11_file = r.TH2F("h_trig11_file", "Trigger 11 vs File", binsx, xmin_2d, xmax_2d, binsy, ymin, ymax)
h_trig13_file = r.TH2F("h_trig13_file", "Trigger 13 vs File", binsx, xmin_2d, xmax_2d, binsy, ymin, ymax)



#add root histogram to plotter
myplotter.addHistograms(h_triggers, 'tTrigger')
myplotter.addHistograms(h_trig1, 'trig1', cut='firsts')
myplotter.addHistograms(h_trig2, 'trig2', cut='firsts')
myplotter.addHistograms(h_trig3, 'trig3', cut='firsts')
myplotter.addHistograms(h_trig4, 'trig4', cut='firsts')
myplotter.addHistograms(h_trig5, 'trig5', cut='firsts')
myplotter.addHistograms(h_trig7, 'trig7', cut='firsts')
myplotter.addHistograms(h_trig9, 'trig9', cut='firsts')
myplotter.addHistograms(h_trig10, 'trig10', cut='firsts')
myplotter.addHistograms(h_trig11, 'trig11', cut='firsts')
myplotter.addHistograms(h_trig13, 'trig13', cut='firsts')

myplotter.addHistograms(h_trig1_file, ["runNumber", "trig1"], cut='firsts')
myplotter.addHistograms(h_trig2_file, ["runNumber", "trig2"], cut='firsts')
myplotter.addHistograms(h_trig3_file, ["runNumber", "trig3"], cut='firsts')
myplotter.addHistograms(h_trig4_file, ["runNumber", "trig4"], cut='firsts')
myplotter.addHistograms(h_trig5_file, ["runNumber", "trig5"], cut='firsts')
myplotter.addHistograms(h_trig7_file, ["runNumber", "trig7"], cut='firsts')
myplotter.addHistograms(h_trig9_file, ["runNumber", "trig9"], cut='firsts')
myplotter.addHistograms(h_trig10_file, ["runNumber", "trig10"], cut='firsts')
myplotter.addHistograms(h_trig11_file, ["runNumber", "trig11"], cut='firsts')
myplotter.addHistograms(h_trig13_file, ["runNumber", "trig13"], cut='firsts')


#defining the cutflow
#cutflow = [boardMatchCut, pickupCut, 
cutflow = [trig1, trig2, trig3, trig4, trig5, trig7, trig9, trig10, trig11, trig13,
            mycuts.firstEvent, myplotter.dict['h_triggers'], 
            myplotter.dict['h_trig1'], myplotter.dict['h_trig2'], myplotter.dict['h_trig3'], myplotter.dict['h_trig4'], 
            myplotter.dict['h_trig5'], myplotter.dict['h_trig7'], myplotter.dict['h_trig9'], myplotter.dict['h_trig10'], 
            myplotter.dict['h_trig11'], myplotter.dict['h_trig13'], myplotter.dict['h_trig1_file'], myplotter.dict['h_trig2_file'],
            myplotter.dict['h_trig3_file'], myplotter.dict['h_trig4_file'], myplotter.dict['h_trig5_file'], myplotter.dict['h_trig7_file'],
            myplotter.dict['h_trig9_file'], myplotter.dict['h_trig10_file'], myplotter.dict['h_trig11_file'], myplotter.dict['h_trig13_file']]

#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

#create the milliqan processor object
myiterator = milliqanProcessor(filelist, branches, myschedule, step_size=1000)

#run the milliqan processor
myiterator.run()

#save plots
myplotter.saveHistograms("test.root")

['/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.185_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.186_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.187_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.188_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.189_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.18_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.190_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.191_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.192_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1401.193_v34.root']
----------------------------
MilliQan Scheduler:
	0. trig1
	1. trig2
	2. trig3
	3. trig4
	4. trig5
	5. trig7
	6. trig9
	7. trig10
	8. trig11
	9. trig13
	10. firstEvent
	11. h_triggers
	12. h_trig1
	13. h_trig2
	14. h_trig3
	15. h_trig4
	16. h_trig5
	17. h_trig7
	18. h_trig9
	19. h_trig10
	20. h_trig11
	21. h_

Warning in <TROOT::Append>: Replacing existing TH1: h_triggers (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig3 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig4 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig5 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig7 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig9 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig10 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig11 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig13 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig1

In [6]:
c1 = r.TCanvas("c1", "c1", 1000, 600)

In [3]:
fin = r.TFile.Open("triggerRateStudy_april19.root", 'read')
fin.ls()
h_trig1 = fin.Get("h_trig1")
h_trig2 = fin.Get("h_trig2")
h_trig3 = fin.Get("h_trig3")
h_trig4 = fin.Get("h_trig4")
h_trig5 = fin.Get("h_trig5")
h_trig7 = fin.Get("h_trig7")
h_trig9 = fin.Get("h_trig9")
h_trig10 = fin.Get("h_trig10")
h_trig11 = fin.Get("h_trig11")
h_trig13 = fin.Get("h_trig13")

h_trig1_file = fin.Get("h_trig1_file")
h_trig2_file = fin.Get("h_trig2_file")
h_trig3_file = fin.Get("h_trig3_file")
h_trig4_file = fin.Get("h_trig4_file")
h_trig5_file = fin.Get("h_trig5_file")
h_trig7_file = fin.Get("h_trig7_file")
h_trig9_file = fin.Get("h_trig9_file")
h_trig10_file = fin.Get("h_trig10_file")
h_trig11_file = fin.Get("h_trig11_file")
h_trig13_file = fin.Get("h_trig13_file")

OSError: Failed to open file triggerRateStudy_april19.root

Error in <TFile::TFile>: file /share/scratch0/mcarrigan/milliQan/analysis/milliqanOffline/Run3Detector/analysis/wip/triggerRateStudy_april19.root does not exist


In [7]:
h_trig1_file.Draw("colz")
h_trig1_file.GetXaxis().SetTitle("Run Number")
h_trig1_file.GetYaxis().SetTitle("Number of Triggers")
c1.Draw()

In [13]:
h_trig2_file.Draw("colz")
h_trig2_file.GetXaxis().SetTitle("Run Number")
h_trig2_file.GetYaxis().SetTitle("Number of Triggers")
c1.Draw()

In [14]:
h_trig3_file.Draw("colz")
h_trig3_file.GetXaxis().SetTitle("Run Number")
h_trig3_file.GetYaxis().SetTitle("Number of Triggers")
c1.Draw()

In [15]:
h_trig4_file.Draw("colz")
h_trig4_file.GetXaxis().SetTitle("Run Number")
h_trig4_file.GetYaxis().SetTitle("Number of Triggers")
c1.Draw()

In [16]:
h_trig5_file.Draw("colz")
h_trig5_file.GetXaxis().SetTitle("Run Number")
h_trig5_file.GetYaxis().SetTitle("Number of Triggers")
c1.Draw()

In [17]:
h_trig7_file.Draw("colz")
h_trig7_file.GetXaxis().SetTitle("Run Number")
h_trig7_file.GetYaxis().SetTitle("Number of Triggers")
c1.Draw()

In [18]:
h_trig9_file.Draw("colz")
h_trig9_file.GetXaxis().SetTitle("Run Number")
h_trig9_file.GetYaxis().SetTitle("Number of Triggers")
c1.Draw()

In [19]:
h_trig10_file.Draw("colz")
h_trig10_file.GetXaxis().SetTitle("Run Number")
h_trig10_file.GetYaxis().SetTitle("Number of Triggers")
c1.Draw()

In [20]:
h_trig11_file.Draw("colz")
h_trig11_file.GetXaxis().SetTitle("Run Number")
h_trig11_file.GetYaxis().SetTitle("Number of Triggers")
c1.Draw()

In [21]:
h_trig13_file.Draw("colz")
h_trig13_file.GetXaxis().SetTitle("Run Number")
h_trig13_file.GetYaxis().SetTitle("Number of Triggers")
c1.Draw()

In [22]:
h_triggers.Draw()
c1.Draw()

In [16]:
'''fin = r.TFile.Open("triggerRateStudy.root", 'read')
fin.ls()
h_trig1 = fin.Get("h_trig1")
h_trig2 = fin.Get("h_trig2")
h_trig3 = fin.Get("h_trig3")
h_trig4 = fin.Get("h_trig4")
h_trig5 = fin.Get("h_trig5")
h_trig7 = fin.Get("h_trig7")
h_trig9 = fin.Get("h_trig9")
h_trig10 = fin.Get("h_trig10")
h_trig11 = fin.Get("h_trig11")
h_trig13 = fin.Get("h_trig13")'''

TFile**		triggerRateStudy.root	
 TFile*		triggerRateStudy.root	
  KEY: TH1F	h_trig1;1	Trigger 1
  KEY: TH1F	h_trig2;1	Trigger 2
  KEY: TH1F	h_trig3;1	Trigger 3
  KEY: TH1F	h_trig4;1	Trigger 4
  KEY: TH1F	h_trig5;1	Trigger 5
  KEY: TH1F	h_trig7;1	Trigger 7
  KEY: TH1F	h_trig9;1	Trigger 9
  KEY: TH1F	h_trig10;1	Trigger 10
  KEY: TH1F	h_trig11;1	Trigger 11
  KEY: TH1F	h_trig13;1	Trigger 13


In [23]:
parameters = {}

h_trig1.Draw()
f1 = r.TF1("f1", "gaus", 1, 100)
h_trig1.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig1'] = [round(mean, 2), round(sigma, 2)]
c1.Draw()

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      3054.38
NDf                       =           26
Edm                       =   2.7538e-08
NCalls                    =          137
Constant                  =       1319.5   +/-   18.4064     
Mean                      =      24.6587   +/-   0.0651133   
Sigma                     =      6.00747   +/-   0.0582131    	 (limited)


In [24]:
h_trig2.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 100, 300)
h_trig2.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig2'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      3585.84
NDf                       =          144
Edm                       =  1.31586e-07
NCalls                    =          151
Constant                  =      445.504   +/-   5.50346     
Mean                      =      195.763   +/-   0.174044    
Sigma                     =      16.7744   +/-   0.116506     	 (limited)


In [25]:
h_trig3.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 1, 100)
h_trig3.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig3'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      4153.22
NDf                       =           67
Edm                       =   8.3917e-09
NCalls                    =          201
Constant                  =      932.528   +/-   13.1258     
Mean                      =      43.9151   +/-   0.0790571   
Sigma                     =      7.29915   +/-   0.0656803    	 (limited)


In [26]:
h_trig4.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 1, 100)
h_trig4.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig4'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      4414.19
NDf                       =           65
Edm                       =  2.97778e-08
NCalls                    =          137
Constant                  =      855.595   +/-   12.079      
Mean                      =      54.8884   +/-   0.0866853   
Sigma                     =      7.93022   +/-   0.0716137    	 (limited)


In [27]:
h_trig5.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 1, 50)
h_trig5.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig5'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      2296.96
NDf                       =           51
Edm                       =  8.55133e-07
NCalls                    =          597
Constant                  =      2284.98   +/-   28.4472     
Mean                      =      6.17969   +/-   0.0660021   
Sigma                     =      3.88754   +/-   0.0440037    	 (limited)


In [28]:
h_trig7.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 50, 150)
h_trig7.Fit("f1", "0", "0", 50, 150)
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig7'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      736.597
NDf                       =           47
Edm                       =  1.04671e-05
NCalls                    =           83
Constant                  =      604.779   +/-   8.59074     
Mean                      =      97.5853   +/-   0.12722     
Sigma                     =      11.1556   +/-   0.101923     	 (limited)


In [29]:
h_trig9.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 1, 150)
h_trig9.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig9'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      3663.23
NDf                       =           54
Edm                       =  7.50603e-06
NCalls                    =          249
Constant                  =      607.629   +/-   7.86747     
Mean                      =      59.5945   +/-   0.134488    
Sigma                     =      12.1052   +/-   0.0930316    	 (limited)


In [30]:
h_trig10.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 100, 350)
h_trig10.Fit("f1", "0", "0", 100, 350)
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig10'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      4722.62
NDf                       =           92
Edm                       =  1.83481e-07
NCalls                    =           79
Constant                  =      99.9412   +/-   1.59883     
Mean                      =      209.135   +/-   0.600152    
Sigma                     =      38.8503   +/-   0.296863     	 (limited)


In [31]:
h_trig11.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 1, 100)
h_trig11.Fit("f1", "0", "0", 1, 50)
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig11'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      3306.48
NDf                       =           18
Edm                       =  2.00922e-06
NCalls                    =           94
Constant                  =      1167.91   +/-   15.0895     
Mean                      =      18.7376   +/-   0.0697932   
Sigma                     =       6.6369   +/-   0.0544871    	 (limited)


In [32]:
h_trig13.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 100, 250)
h_trig13.Fit("f1","0", "0", 120, 220)
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig13'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      170.148
NDf                       =           46
Edm                       =  2.98053e-06
NCalls                    =           57
Constant                  =      445.161   +/-   5.91978     
Mean                      =      171.738   +/-   0.163928    
Sigma                     =      14.7593   +/-   0.111069     	 (limited)


In [11]:
'''fout = r.TFile.Open("triggerRateStudy_april19.root", 'recreate')

fout.cd()
h_trig1.Write()
h_trig2.Write()
h_trig3.Write()
h_trig4.Write()
h_trig5.Write()
h_trig7.Write()
h_trig9.Write()
h_trig10.Write()
h_trig11.Write()
h_trig13.Write()

h_trig1_file.Write()
h_trig2_file.Write()
h_trig3_file.Write()
h_trig4_file.Write()
h_trig5_file.Write()
h_trig7_file.Write()
h_trig9_file.Write()
h_trig10_file.Write()
h_trig11_file.Write()
h_trig13_file.Write()

fout.Close()'''

In [33]:
h_trig2.SetLineColor(2)
h_trig3.SetLineColor(3)
h_trig4.SetLineColor(4)
h_trig5.SetLineColor(6)
h_trig7.SetLineColor(7)
h_trig9.SetLineColor(46)
h_trig10.SetLineColor(51)
h_trig11.SetLineColor(23)
h_trig13.SetLineColor(94)

h_trig1.SetTitle("Triggers Fired per File (Runs 1400+)")
h_trig1.GetXaxis().SetTitle("Triggers per File")
h_trig1.GetYaxis().SetTitle("Number of Files")

h_trig1.Draw()
h_trig2.Draw("same")
h_trig3.Draw("same")
h_trig4.Draw("same")
h_trig5.Draw("same")
h_trig7.Draw("same")
h_trig9.Draw("same")
h_trig10.Draw("same")
h_trig11.Draw("same")
h_trig13.Draw("same")

l1 = r.TLegend(0.6, 0.9, 0.6, 0.9)
l1.AddEntry(h_trig1, "Trigger 1: \mu {} \pm {}".format(parameters['trig1'][0], parameters['trig1'][1]), 'l')
l1.AddEntry(h_trig2, "Trigger 2: \mu {} \pm {}".format(parameters['trig2'][0], parameters['trig2'][1]), 'l')
l1.AddEntry(h_trig3, "Trigger 3: \mu {} \pm {}".format(parameters['trig3'][0], parameters['trig3'][1]), 'l')
l1.AddEntry(h_trig4, "Trigger 4: \mu {} \pm {}".format(parameters['trig4'][0], parameters['trig4'][1]), 'l')
l1.AddEntry(h_trig5, "Trigger 5: \mu {} \pm {}".format(parameters['trig5'][0], parameters['trig5'][1]), 'l')
l1.AddEntry(h_trig7, "Trigger 7: \mu {} \pm {}".format(parameters['trig7'][0], parameters['trig7'][1]), 'l')
l1.AddEntry(h_trig9, "Trigger 9: \mu {} \pm {}".format(parameters['trig9'][0], parameters['trig9'][1]), 'l')
l1.AddEntry(h_trig10, "Trigger 10: \mu {} \pm {}".format(parameters['trig10'][0], parameters['trig10'][1]), 'l')
l1.AddEntry(h_trig11, "Trigger 11: \mu {} \pm {}".format(parameters['trig11'][0], parameters['trig11'][1]), 'l')
l1.AddEntry(h_trig13, "Trigger 13: \mu {} \pm {}".format(parameters['trig13'][0], parameters['trig13'][1]), 'l')


l1.Draw()
c1.Draw()



Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used
